<a href="https://colab.research.google.com/github/tsato-code/bert/blob/main/20220324_masked_language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# このノートブックの概要

- ストックマーク社の BERT 本5章を動作確認。
- 内容は文章の穴埋め。

TODO
- そもそも文章の穴埋めをしたいのはどのようなときか。

In [1]:
### fugashi は 形態素解析ツール Mecab を Python から使えるようにしたもの
### ipadic は Mecab で形態素解析を利用するときに使う辞書
!pip install -q transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 5.0 MB/s 
     |████████████████████████████████| 486 kB 35.8 MB/s 
     |████████████████████████████████| 13.4 MB 57.4 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 3.3 MB 50.8 MB/s 


In [2]:
### グローバル変数
JAPANESE_WIKI_MODEL = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [3]:
### ライブラリのインポート
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [5]:
### 準備
tokenizer = BertJapaneseTokenizer.from_pretrained(JAPANESE_WIKI_MODEL)
bert_mlm = BertForMaskedLM.from_pretrained(JAPANESE_WIKI_MODEL)
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
text = '[MASK]の夜明けぜよ'
tokens = tokenizer.tokenize(text)
print(tokens)

['[MASK]', 'の', '夜明け', 'ぜ', 'よ']


In [64]:
### 符号化
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

### 分類スコア
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
scores = output.logits

print(scores.shape)
print(scores)

torch.Size([1, 12, 32000])
tensor([[[-4.8918,  4.6949, -3.5190,  ..., -4.3858, -3.9782, -2.9297],
         [-4.2450,  5.8047, -3.1998,  ..., -3.4704, -6.0453, -3.4959],
         [-3.6191,  7.0827, -3.8722,  ..., -6.0576, -6.2808, -6.6423],
         ...,
         [-4.3523,  5.3218, -3.8007,  ..., -4.0504, -3.5354, -2.5074],
         [-4.1096,  5.1667, -3.7727,  ..., -3.7396, -3.7238, -2.1865],
         [-6.4411,  6.2043, -6.2109,  ..., -3.5276, -5.3214, -7.1897]]],
       device='cuda:0')


In [24]:
### [MASK] の位置
mask_position = input_ids[0].tolist().index(4)
print(mask_position)

### 最大スコアのトークン id をもとにトークンに変換
id_best = scores[0, mask_position].argmax().item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')
print(token_best)

### 元文を置換
print(text.replace('[MASK]', token_best))

1
明日
明日の夜明けぜよ


In [34]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    ### 符号化
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()

    ### 分類スコア
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits
    
    ### スコア上位のトークン id をもとにトークンに変換
    mask_position = input_ids[0].tolist().index(4)
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
    scores_topk = topk.values.cpu().numpy()

    ### スコア上位のトークンで元文を置換
    text_topk = []
    for token in tokens_topk:
        token = token.replace('##', '')
        ### print(token)
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

text = '知床の海に[MASK]'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

知床の海に浮かぶ
知床の海に沈む
知床の海に。
知床の海にある
知床の海に面する
知床の海に...
知床の海に面し
知床の海に立つ
知床の海に[UNK]
知床の海に架かる


In [39]:
### 複数の [MASK] を前から貪欲法で埋める
def greedy_prediction(text, tokenizer, bert_mlm):
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text


text = '[MASK]の[MASK]に[MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'社会のために。'

In [60]:
### [MASK] が多すぎると苦手
text = '個人情報保護法が[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'個人情報保護法が社会的に成立。'

In [63]:
### 複数の [MASK] をビームサーチで埋める
def beam_search(text, tokenizer, bert_mlm, num_topk):
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        text_candidates = []
        score_candidates = []
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk)
            text_candidates.extend( text_topk_inner )
            score_candidates.append( score + scores_topk_inner )
        
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [text_candidates[idx] for idx in idx_list]
        scores_topk = score_candidates[idx_list]
    
    return text_topk


text_topk = beam_search(text, tokenizer, bert_mlm, 20)
print(*text_topk, sep='\n')

個人情報保護法が社会化される。
個人情報保護法が社会に広がった。
個人情報保護法が社会問題となる。
個人情報保護法が社会問題化する。
個人情報保護法が社会問題になる。
個人情報保護法が社会に広まる。
個人情報保護法が社会化された
個人情報保護法が社会問題となった
個人情報保護法が社会に広まった。
個人情報保護法が社会化した。
個人情報保護法が2位である。
個人情報保護法が社会問題になった
個人情報保護法が社会問題に発展。
個人情報保護法が社会問題化した
個人情報保護法が社会化され、
個人情報保護法が社会化され。
個人情報保護法が社会問題となり、
個人情報保護法が社会化されると
個人情報保護法が社会問題となり。
個人情報保護法が社会に広まり。
